# Load Individual Household Electric Power Consumption
https://archive.ics.uci.edu/dataset/235/individual+household+electric+power+consumption

## Readme
- https://medium.com/@ottaviocalzone/an-intuitive-explanation-of-lstm-a035eb6ab42c
- https://www.youtube.com/watch?v=b61DPVFX03I

In [ ]:
import os
import torch
from support import EnergyConsumptionDataset, EnergyConsumptionModule, build_loaders, demo_model_shapes, \
    plot_convergence, train_and_evaluate, plot_predictions_grid, plot_mse_horizon

In [ ]:
data_folder = "local_data/cache"
if not os.path.exists(data_folder):
    os.makedirs(data_folder, exist_ok=True)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

In [ ]:
all_data = EnergyConsumptionDataset(history_samples=24 * 14, horizon_samples=24 * 3, scale=True, train_fraction=0.9)
print("data length: ", len(all_data))
print("number of channels: ", all_data.number_of_channels)

# Explore dataset

In [ ]:
all_data.data_features.head()

In [ ]:
train_dataset, test_dataset, train_loader, test_loader = build_loaders(all_data, train_fraction=0.9, batch_size_train=200, batch_size_eval=2000)

In [ ]:
model = EnergyConsumptionModule(input_dim=all_data.number_of_channels, output_dim=all_data.horizon_samples)

In [ ]:
demo_model_shapes(model, train_loader)

In [ ]:
checkpoint_file_name = os.path.join(data_folder, "ER Admission - CNN-LSTM Fusion.checkpoint.pt")
model, progress_log, test_accuracy, test_loader = train_and_evaluate(model, train_dataset, test_dataset, device=device, n_epochs=10, batch_size=200, lr=1e-3, checkpoint_file_name=checkpoint_file_name)
plot_convergence(progress_log, test_accuracy)
plot_predictions_grid(model, test_loader, device=device, n=6)
plot_mse_horizon(model, test_loader, all_data.horizon_samples, device=device)